In [19]:
import pandas as pd
import openai
import time
import random

In [9]:
# 3. CSV 파일 불러오기
df = pd.read_csv("../dataset/daily.csv")
print(df[0:10])

       증상
0      건선
1   하지정맥류
2    장티푸스
3      수두
4     농가진
5     뎅기열
6  곰팡이 감염
7      감기
8      폐렴
9   혼합 치질


In [14]:
print(df[:-1])

                                                     증상
0                                                    건선
1                                                 하지정맥류
2                                                  장티푸스
3                                                    수두
4                                                   농가진
...                                                 ...
3413                                B형 간염 [hepatitis B]
3414                                C형 간염 [hepatitis C]
3415                                D형 간염 [hepatitis D]
3416                          LCP병 [Legg-Calve-Perthes]
3417  Rh 신생아 용혈성 질환 [Rh hemolytic disease of the new...

[3418 rows x 1 columns]


# 오타 정의

In [22]:
def random_space_typo(text, prob=0.15):
    words = text.split()
    new_text = ""
    for i, w in enumerate(words):
        new_text += w
        # 띄어쓰기 실수(확률적으로 붙임)
        if i != len(words) - 1:
            if random.random() > prob:
                new_text += " "
    return new_text


# 아주 간단한 철자 오타(받침 탈락 등)
def random_simple_typo(text, prob=0.07):
    typo_dict = {
        # 자주 쓰는 모음/자음 실수, 채팅체/애교체
        "아": "어",
        "어": "아",
        "에": "애",
        "애": "에",
        "이": "잉",
        "니": "닝",  # 애교체
        "요": "용",
        "야": "양",
        "우": "으",
        "으": "우",
        "는": "늘",
        "다": "따",
        "라": "나",
        "나": "라",
        "거": "꺼",
        "게": "께",
        # 단어 단위(많이 쓰는 단어 변형)
        "좀": "쫌",
        "진짜": "찐짜",
        "너무": "느무",
        "아니": "아닝",
        "그래": "글애",
        "그래서": "글애서",
        "근데": "근대",
        "이렇게": "일케",
        "몰라": "몰랑",
        "좋아": "조아",
        "맞아": "맞아용",
        "싫어": "실허",
        "귀찮아": "귀차나",
        "많이": "마니",
        "빨리": "빨리",
        "모르겠어": "모르겟어",
        "어려워": "어려웡",
        "싶어": "시퍼",
        "같아": "같애",
        "있어": "이써",
        "없어": "업써",
        "합니다": "함니다",
        "합니당": "함니당",
        "해요": "해용",
        "하네": "하넹",
        "봐": "바",
        "봐요": "바용",
        "라고": "라구",
        "하고": "하구",
        "싶다": "시따",
        "이상해": "이상행",
        "오늘": "오눌",
        "좋아요": "조아용",
        "지금": "즁금",
        "랗": "라",  # 받침 탈락
    }

    result = ""
    for char in text:
        if char in typo_dict and random.random() < prob:
            result += typo_dict[char]
        else:
            result += char
    return result

# 코드 (OpenAI GPT-4o기준)
- test(10개 정도)

In [ ]:
# 1. OpenAI API 키 입력
openai.api_key = "gpt key"


# 2. 프롬프트 설계
def symptom_to_daily(text):
    if pd.isnull(text) or not str(text).strip():
        return ""
    prompt = (
        f"아래 병명에 대해, 실제 환자가 그 병에 걸렸다고 가정하고 "
        f"해당 병의 대표 증상이나 특징적인 증상을 중심으로 "
        f"실제로 사람들이 자주 쓰는 일상적인 말투(반말, 줄임말, 띄어쓰기 약간 틀려도 됨)와 "
        f"의성어·의태어(예: 욱신욱신, 따끔따끔, 저릿저릿 등)를 활용해서 "
        f"병명(증상명)은 절대 문장 안에 직접 쓰지 말고, "
        f"실제로 사람들이 메시지 보낼 때처럼 띄어쓰기 실수나 오타, 줄임말이 자연스럽게 들어가도 되고, "
        f"너무 딱딱하거나 교과서적이지 않게, 평소 말하는 느낌으로 만들어줘. "
        f"짧고 구체적인 한 문장(질문 형태나 걱정하는 형태도 가능)으로 바꿔줘. "
        f'즉, "증상을 호소하는 구어체 문장"만 생성하고, '
        f"병명은 문장 안에 절대 넣지 마. "
        f"실수로 오타, 띄어쓰기 오류, 말 줄임, 중간에 끊긴 문장 등도 일부러 자연스럽게 만들어줘."
        f"예시:\n"
        f'"감기" → "콧물은 질질, 콜록콜록 기침도 나고 목도 칼칼한데 이거 뭐야?",\n'
        f'"관절염" → "손가락이 욱신욱신 아프고 뻣뻣해서 젓가락질도 힘든데 큰일난거야?",\n'
        f'"피부염" → "따끔따끔 간지러워서 긁고 싶은데 자꾸 더 심해지는 것 같아. 왜 이러지?"\n'
        f"반드시 위 예시처럼 의성어나 의태어(콜록콜록, 욱신욱신, 따끔따끔, 뻣뻣, 쿨럭쿨럭 등)를 꼭 한 번 이상"
        f'입력: "{text}"'
    )

    # GPT API 호출
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o", messages=[{"role": "user", "content": prompt}]
            )
            return response.choices[0].message["content"].strip()
        except Exception as e:
            print(f"에러 발생, 5초 후 재시도: {e}")
            time.sleep(5)


# 3. CSV 파일 불러오기 (첫 줄이 컬럼명/헤더라고 가정)
df = pd.read_csv("../dataset/daily.csv")

# 4. A2-A11 총 10개 테스트
df_target = df.iloc[0:20].copy()

# 5. 일상말 컬럼(B열) 생성
df_target["일상말"] = df_target["증상"].apply(symptom_to_daily)

# === [추가] 오타/띄어쓰기 후처리 ===
df_target["일상말"] = df_target["일상말"].apply(random_space_typo)
df_target["일상말"] = df_target["일상말"].apply(random_simple_typo)

# === [추가] 따옴표, 작은따옴표 제거 ===
df_target["일상말"] = df_target["일상말"].str.replace('"', "", regex=False)
df_target["일상말"] = df_target["일상말"].str.replace("'", "", regex=False)

# 6. 결과 파일 저장 (헤더: 증상, 일상말)
df_target.to_csv("daily_test.csv", index=False, encoding="utf-8-sig")

print("변환 완료! 결과 파일: daily_test.csv")

변환 완료! 결과 파일: daily_test.csv


In [ ]:
# 1. OpenAI API 키 입력
openai.api_key = "gpt key"


# 2. 프롬프트 설계
def symptom_to_daily(text):
    if pd.isnull(text) or not str(text).strip():
        return ""
    prompt = (
        f"아래 병명에 대해, 실제 환자가 그 병에 걸렸다고 가정하고 "
        f"해당 병의 대표 증상이나 특징적인 증상을 중심으로 "
        f"실제로 사람들이 자주 쓰는 일상적인 말투(반말, 줄임말, 띄어쓰기 약간 틀려도 됨)와 "
        f"의성어·의태어(예: 욱신욱신, 따끔따끔, 저릿저릿 등)를 활용해서 "
        f"병명(증상명)은 절대 문장 안에 직접 쓰지 말고, "
        f"실제로 사람들이 메시지 보낼 때처럼 띄어쓰기 실수나 오타, 줄임말이 자연스럽게 들어가도 되고, "
        f"너무 딱딱하거나 교과서적이지 않게, 평소 말하는 느낌으로 만들어줘. "
        f"짧고 구체적인 한 문장(질문 형태나 걱정하는 형태도 가능)으로 바꿔줘. "
        f'즉, "증상을 호소하는 구어체 문장"만 생성하고, '
        f"병명은 문장 안에 절대 넣지 마. "
        f"실수로 오타, 띄어쓰기 오류, 말 줄임, 중간에 끊긴 문장 등도 일부러 자연스럽게 만들어줘."
        f"예시:\n"
        f'"감기" → "콧물은 질질, 콜록콜록 기침도 나고 목도 칼칼한데 이거 뭐야?",\n'
        f'"관절염" → "손가락이 욱신욱신 아프고 뻣뻣해서 젓가락질도 힘든데 큰일난거야?",\n'
        f'"피부염" → "따끔따끔 간지러워서 긁고 싶은데 자꾸 더 심해지는 것 같아. 왜 이러지?"\n'
        f"반드시 위 예시처럼 의성어나 의태어(콜록콜록, 욱신욱신, 따끔따끔, 뻣뻣, 쿨럭쿨럭 등)를 꼭 한 번 이상"
        f'입력: "{text}"'
    )

    # GPT API 호출
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o", messages=[{"role": "user", "content": prompt}]
            )
            return response.choices[0].message["content"].strip()
        except Exception as e:
            print(f"에러 발생, 5초 후 재시도: {e}")
            time.sleep(5)


# 3. CSV 파일 불러오기 (첫 줄이 컬럼명/헤더라고 가정)
df = pd.read_csv("../dataset/daily.csv")

# 4. A2-A3420 전체
df_target = df.iloc[0:-1].copy()

# 5. 일상말 컬럼(B열) 생성
df_target["일상말"] = df_target["증상"].apply(symptom_to_daily)

# === [추가] 오타/띄어쓰기 후처리 ===
df_target["일상말"] = df_target["일상말"].apply(random_space_typo)
df_target["일상말"] = df_target["일상말"].apply(random_simple_typo)

# === [추가] 따옴표, 작은따옴표 제거 ===
df_target["일상말"] = df_target["일상말"].str.replace('"', "", regex=False)
df_target["일상말"] = df_target["일상말"].str.replace("'", "", regex=False)

# 6. 결과 파일 저장 (헤더: 증상, 일상말)
df_target.to_csv("daily_dataset.csv", index=False, encoding="utf-8-sig")

print("변환 완료! 결과 파일: daily_dataset.csv")

변환 완료! 결과 파일: daily_dataset.csv
